In [ ]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd

df3 = pd.read_csv('drive/MyDrive/지하철/서울교통공사_2023년.csv', encoding='cp949')
station = dict() #역명을 숫자로 교체 (서울역 = 1 , 용산 = 2 ...)
sdx = 0
station_list = [] #역명 담을 리스트
subway_list = [] #지하철 호선 담을 리스트
week_list = [] #요일 담을 리스트(월요일 = 0 , 화요일 = 1 , 수요일 = 2 ...)


#------------------------ 2023년--------------------
#2023년 1월 1일은 일요일 -> 6
prev = "2023-01-01"
idx = 6
cnt = 0
for day , subway , st in zip(df3['수송일자'] ,df3['호선'] ,df3['역명']):
  if cnt % 2 == 0: #승차 , 하차 중 1번만 저장하기 위해서 조건문으로 해결함.
    if prev == day: #날짜가 바뀌지 않았을 때
      week_list.append(idx)
    else: #날짜가 바뀌었을 때
      idx = (idx + 1) % 7 #날짜가 증가하였기 때문에 요일도 증가
      week_list.append(idx)
      prev = day
    subway_list.append(int(subway[0])) #인덱스 0번을 숫자로 변형
    if st in station: #역명이 이전에 존재했던 경우
      station_list.append(station[st])
    else: #역명이 이전에 존재하지 않았을 경우 숫자로 변형
      station[st] = sdx
      station_list.append(sdx)
      sdx += 1
  cnt += 1


#------------------------ 2022년--------------------
df2 = pd.read_csv('drive/MyDrive/지하철/서울교통공사_2022년.csv', encoding='cp949')
#2022년 1월 1일은 토요일 -> 5
prev = "2022-01-01"
idx = 5
cnt = 0
for day , subway , st in zip(df2['수송일자'] ,df2['호선'] ,df2['역명']):
  if cnt % 2 == 0:
    if prev == day:
      week_list.append(idx)
    else:
      idx = (idx + 1) % 7
      week_list.append(idx)
      prev = day
    subway_list.append(subway)
    if st in station:
      station_list.append(station[st])
    else:
      station[st] = sdx
      station_list.append(sdx)
      sdx += 1
  cnt += 1


#------------------------ 2021년--------------------
df1 = pd.read_csv('drive/MyDrive/지하철/서울교통공사_2021년.csv', encoding='cp949')
#2022년 1월 1일은 금요일 -> 4
prev = "2021-01-01"
idx = 4
cnt = 0
for day , subway , st in zip(df1['날짜'] ,df1['호선'] ,df1['역명']):
  if cnt % 2 == 0:
    if prev == day:
      week_list.append(idx)
    else:
      idx = (idx + 1) % 7
      week_list.append(idx)
      prev = day
    subway_list.append(subway)
    if st in station:
      station_list.append(station[st])
    else:
      station[st] = sdx
      station_list.append(sdx)
      sdx += 1
  cnt += 1


X = dict()
X['역명'] = station_list
X['호선'] = subway_list
X['요일'] = week_list
X = DataFrame(X)




<ipython-input-59-24540bf25378>:37: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('drive/MyDrive/지하철/서울교통공사_2022년.csv', encoding='cp949')


In [ ]:
X

,역명,호선,요일
0,0,1,6
1,1,1,6
2,2,1,6
3,3,1,6
4,4,1,6
...,...,...,...
234364,234,8,4
234365,235,8,4
234366,236,8,4
234367,237,8,4


In [ ]:
def y_train_select(time):
  t23 = df3[time]
  t22 = df2[time]
  t21 = df1[time]
  t = list(t23) + list(t22) + list(t21)
  array = []
  cnt = 0
  for a in t:
    if cnt % 2 == 0: #승차와 하차 인원수를 합쳐주기 위해서 조건문으로 해결함.
      array.append(a)
    else:
      array[-1] += a
    cnt += 1
  y = dict()
  y['인원수'] = np.log(np.array(array) + 1) #특이값 노이즈를 줄이기 위해서 자연로그를 사용함.
  return y


times = ['06-07시간대' , '07-08시간대' , '08-09시간대' , '17-18시간대' , '18-19시간대' , '19-20시간대' ]
y = DataFrame(y_train_select(times[4])) #18-19시간대

In [ ]:
y

,인원수
0,8.407378
1,7.273786
2,7.515889
3,7.336286
4,7.180831
...,...
234364,7.400010
234365,6.635947
234366,6.620073
234367,6.549651


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
gbm = GradientBoostingRegressor(n_estimators = 4000 , learning_rate = 0.25 , max_depth = 3) #모델 생성
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #학습 데이터셋과 테스트 데이터셋 분리
gbm.fit(X_train , y_train) #모델 학습

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(learning_rate=0.25, n_estimators=4000)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

pred = gbm.predict(X_test) #테스트 데이터셋의 예측값
rmse = sqrt(mean_squared_error(pred , y_test)) #RMSE

print(f"RMSE : {rmse}")

days = ['월요일' , '화요일' , '수요일' , '목요일' , '금요일' , '토요일' , '일요일']
for i in range(7):
  x = DataFrame({'역명' : [0] , '호선' : [1] , '요일' : [i]}) # 월요일 서울역 1호선
  result = int(np.exp(gbm.predict(x)) - 1)
  print(f"{days[i]} 서울역 1호선은 18-19시간대에 {result} 명의 사람들이 있을 예정입니다. ")

RMSE : 0.30210543164215803
월요일 서울역 1호선은 18-19시간대에 9237 명의 사람들이 있을 예정입니다. 
화요일 서울역 1호선은 18-19시간대에 9983 명의 사람들이 있을 예정입니다. 
수요일 서울역 1호선은 18-19시간대에 9843 명의 사람들이 있을 예정입니다. 
목요일 서울역 1호선은 18-19시간대에 10400 명의 사람들이 있을 예정입니다. 
금요일 서울역 1호선은 18-19시간대에 11076 명의 사람들이 있을 예정입니다. 
토요일 서울역 1호선은 18-19시간대에 3976 명의 사람들이 있을 예정입니다. 
일요일 서울역 1호선은 18-19시간대에 3851 명의 사람들이 있을 예정입니다. 


<ipython-input-67-24d5d0b1cbb0>:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = int(np.exp(gbm.predict(x)) - 1)
<ipython-input-67-24d5d0b1cbb0>:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = int(np.exp(gbm.predict(x)) - 1)
<ipython-input-67-24d5d0b1cbb0>:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = int(np.exp(gbm.predict(x)) - 1)
<ipython-input-67-24d5d0b1cbb0>:12: DeprecationWarning: Conversion of an array with ndim > 0 to a sc

In [ ]:
days = ['월요일' , '화요일' , '수요일' , '목요일' , '금요일' , '토요일' , '일요일']
for i in range(7):
  x = DataFrame({'역명' : [0] , '호선' : [1] , '요일' : [i]}) # 월요일 서울역 1호선
  result = int(np.exp(gbm.predict(x)) - 1)
  print(f"{days[i]} 서울역 1호선은 18-19시간대에 {result} 명의 사람들이 있을 예정입니다. ")

In [ ]:
days = ['월요일' , '화요일' , '수요일' , '목요일' , '금요일' , '토요일' , '일요일']

for i in range(7):
  x = DataFrame({'역명' : [1] , '호선' : [2] , '요일' : [i]}) # 월요일 서울역 1호선
  result = int(np.exp(gbm.predict(x)) - 1)
  print(f"{days[i]} 시청 2호선은 18-19시간대에 {result} 명의 사람들이 있을 예정입니다. ")

월요일 시청 2호선은 18-19시간대에 6416 명의 사람들이 있을 예정입니다. 
화요일 시청 2호선은 18-19시간대에 7243 명의 사람들이 있을 예정입니다. 
수요일 시청 2호선은 18-19시간대에 7123 명의 사람들이 있을 예정입니다. 
목요일 시청 2호선은 18-19시간대에 7259 명의 사람들이 있을 예정입니다. 
금요일 시청 2호선은 18-19시간대에 7280 명의 사람들이 있을 예정입니다. 
토요일 시청 2호선은 18-19시간대에 1263 명의 사람들이 있을 예정입니다. 
일요일 시청 2호선은 18-19시간대에 781 명의 사람들이 있을 예정입니다. 


<ipython-input-68-13e57dfcf583>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = int(np.exp(gbm.predict(x)) - 1)
<ipython-input-68-13e57dfcf583>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = int(np.exp(gbm.predict(x)) - 1)
<ipython-input-68-13e57dfcf583>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = int(np.exp(gbm.predict(x)) - 1)
<ipython-input-68-13e57dfcf583>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar

In [ ]:
print(station['혜화'])

99


In [ ]:
days = ['월요일' , '화요일' , '수요일' , '목요일' , '금요일' , '토요일' , '일요일']

for i in range(7):
  x = DataFrame({'역명' : [99] , '호선' : [4] , '요일' : [i]}) # 월요일 서울역 1호선
  result = int(np.exp(gbm.predict(x)) - 1)
  print(f"{days[i]} 혜화 4호선은 18-19시간대에 {result} 명의 사람들이 있을 예정입니다. ")

월요일 혜화 4호선은 18-19시간대에 5611 명의 사람들이 있을 예정입니다. 
화요일 혜화 4호선은 18-19시간대에 6449 명의 사람들이 있을 예정입니다. 
수요일 혜화 4호선은 18-19시간대에 6670 명의 사람들이 있을 예정입니다. 
목요일 혜화 4호선은 18-19시간대에 7148 명의 사람들이 있을 예정입니다. 
금요일 혜화 4호선은 18-19시간대에 7341 명의 사람들이 있을 예정입니다. 
토요일 혜화 4호선은 18-19시간대에 3980 명의 사람들이 있을 예정입니다. 
일요일 혜화 4호선은 18-19시간대에 2831 명의 사람들이 있을 예정입니다. 


<ipython-input-73-9ff4befb6ab8>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = int(np.exp(gbm.predict(x)) - 1)
<ipython-input-73-9ff4befb6ab8>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = int(np.exp(gbm.predict(x)) - 1)
<ipython-input-73-9ff4befb6ab8>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = int(np.exp(gbm.predict(x)) - 1)
<ipython-input-73-9ff4befb6ab8>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar

In [ ]:
print(station['건대입구'])

20


In [ ]:
days = ['월요일' , '화요일' , '수요일' , '목요일' , '금요일' , '토요일' , '일요일']

for i in range(7):
  x = DataFrame({'역명' : [20] , '호선' : [7] , '요일' : [i]}) # 월요일 서울역 1호선
  result = int(np.exp(gbm.predict(x)) - 1)
  print(f"{days[i]} 건대입구 7호선은 18-19시간대에 {result} 명의 사람들이 있을 예정입니다. ")

월요일 건대입구 7호선은 18-19시간대에 2440 명의 사람들이 있을 예정입니다. 
화요일 건대입구 7호선은 18-19시간대에 2576 명의 사람들이 있을 예정입니다. 
수요일 건대입구 7호선은 18-19시간대에 2601 명의 사람들이 있을 예정입니다. 
목요일 건대입구 7호선은 18-19시간대에 2629 명의 사람들이 있을 예정입니다. 
금요일 건대입구 7호선은 18-19시간대에 2801 명의 사람들이 있을 예정입니다. 
토요일 건대입구 7호선은 18-19시간대에 1614 명의 사람들이 있을 예정입니다. 
일요일 건대입구 7호선은 18-19시간대에 1204 명의 사람들이 있을 예정입니다. 


<ipython-input-74-4c31fda30966>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = int(np.exp(gbm.predict(x)) - 1)
<ipython-input-74-4c31fda30966>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = int(np.exp(gbm.predict(x)) - 1)
<ipython-input-74-4c31fda30966>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = int(np.exp(gbm.predict(x)) - 1)
<ipython-input-74-4c31fda30966>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar